In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os

In [4]:
train_data_dir='emotions/train/'
validation_data_dir='emotions/test/'

In [5]:
# List all files and subdirectories in the train_data_dir
print("Contents of train_data_dir:")
for file_or_dir in os.listdir(train_data_dir):
    print(file_or_dir)

# List all files and subdirectories in the validation_data_dir
print("\nContents of validation_data_dir:")
for file_or_dir in os.listdir(validation_data_dir):
    print(file_or_dir)

Contents of train_data_dir:
angry
disgust
fear
happy
neutral
sad
surprise

Contents of validation_data_dir:
angry
disgust
fear
happy
neutral
sad
surprise


In [6]:
train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					horizontal_flip=True,
					fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(48, 48),
					batch_size=32,
					class_mode='categorical',
					shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(48, 48),
							batch_size=32,
							class_mode='categorical',
							shuffle=True)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']

img, label = train_generator.__next__()

In [8]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

In [9]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                   

In [11]:
train_path = "emotions/train/"
test_path = "emotions/test"

num_train_imgs = 0
for root, dirs, files in os.walk(train_path):
    num_train_imgs += len(files)
    
num_test_imgs = 0
for root, dirs, files in os.walk(test_path):
    num_test_imgs += len(files)

print(num_train_imgs)
print(num_test_imgs)

28709
7178


In [13]:
epochs=100
history=model.fit(train_generator,
                steps_per_epoch=num_train_imgs//32,
                epochs=epochs,
                validation_data=validation_generator,
                validation_steps=num_test_imgs//32)

Epoch 1/100
897/897 [==============================] - 550s 612ms/step - loss: 1.7692 - accuracy: 0.2684 - val_loss: 1.6723 - val_accuracy: 0.3337
Epoch 2/100
897/897 [==============================] - 295s 329ms/step - loss: 1.6449 - accuracy: 0.3431 - val_loss: 1.4582 - val_accuracy: 0.4304
Epoch 3/100
897/897 [==============================] - 174s 194ms/step - loss: 1.5115 - accuracy: 0.4121 - val_loss: 1.3361 - val_accuracy: 0.4908
Epoch 4/100
897/897 [==============================] - 164s 183ms/step - loss: 1.4178 - accuracy: 0.4526 - val_loss: 1.2475 - val_accuracy: 0.5183
Epoch 5/100
897/897 [==============================] - 156s 174ms/step - loss: 1.3575 - accuracy: 0.4800 - val_loss: 1.2407 - val_accuracy: 0.5283
Epoch 6/100
897/897 [==============================] - 164s 183ms/step - loss: 1.3206 - accuracy: 0.4937 - val_loss: 1.1930 - val_accuracy: 0.5420
Epoch 7/100
897/897 [==============================] - 168s 188ms/step - loss: 1.2910 - accuracy: 0.5068 - val_loss: 1

In [15]:
from keras.models import save_model

# Guradamos el modelo para poder usarlo despues
model.save('recofnition_100epochs.h5')

C:\Users\Ricardo Caceres\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
